## Update `trips`
* cd rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env
* https://github.com/cal-itp/data-analyses/pull/1016
    * Keep source data + metrics tightly defined with GCS bucket organization.
    * vp_usable is source data for rt_vs_sched metrics, do not merge in schedule data until gtfs_digest report. Only bring in schedule_gtfs_dataset_key column in    
    * vp_usable + route_id-direction_id for trips also present in schedule. If not in schedule, fill it with route_id = Unknown and direction_id as Int64
    * Add function to concatenate trip file, enable us to put in 1 day or 7 days for aggregation
    * A single function for normalized metrics (percent, per min, etc)
    * A single function for aggregation (summing up numerator / denominator)
    
* https://github.com/cal-itp/data-analyses/issues/989

* Notes 2/6
    * GTFS digest creates four datasets: schedule, average speeds, segment speeds, and rt vs schedule
    * Currently, merging is challenging.
    * Time categories are not necessarily the same (peak/offpeak/all-day)
    * Want all datasets to merge on the same set of columns (schedule gtfs key, route id, dir id, service date, and time categories) because `shapes` are unstable.
    * `Route ID` has been stabilized by Tiffany 
    * Update work from `rt_v_scheduled.py` (steps already outlined in `scripts/route_aggregation.ipynb`)
        * Do steps up until row 339 when the % are calculated. 
        * Take away `speeds`.
        * Bring in schedule gtfs key, trip instance key, route id, direction id either at the beginning or the end using `helpers.import_scheduled_trips`
        * Coerce DIR ID to Int64, don't fill it in with 0. It's not 0, it's Nan
        * Save files with the analysis date at the end instead of the beginning.
        * Split off the workstream -> one for trip level and one for route level
            * Use the config.yml to save the trips and routes stuff into their own folder.
            * Routes:
                * For routes, the minutes/pings should be totalled up. Currently, just taking the average of an average isn't really accurate.
                * The route level should be able to take multiple days of data and concatenate so we can get metrics for a week/2 weeks/etc instead of for a single day. [Done here](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/scripts/average_speeds.py)
                * Add the route frequency as well?
           * Trips:
               * Do up to step 339 in `rt_v_scheduled.py`
               * Write a new generalized function to create all the % 
            
* Notes 2/13
    * Figure out how to set up Config file
    * Tiffany:
        * add_metrics looks good, just remove the coercing of percents to 0-100 to a separate function. I want everything from 0-1, and then before charting, scaled up to 0-100 all at once. Can you write a general         * function for this....all the chart display / cleaning functions should live in 1 script in segment_speed_utils.
        * Another tweak for a step somewhere before add_metrics. Certain columns can be coerced to be integers, like total_vp and vp_in_shape, just like how total_min_w_gtfs is an integer. Coerce all the ones that can be integers to be integers for your trip table, and this will save on the rounding step later.
        * Column naming: think about how you want to change the column names. total_pings_for_trip is not going to make sense once you aggregate, so maybe go with something more generic. Otherwise, you're going to be aggregating and renaming columns constantly. I would just rely on the other columns in the row to tell us whether it's per trip or per route , and the metrics all use generic names that are suitable for passing through aggregation functions. (edited) 

In [1]:
import dask.dataframe as dd
import pandas as pd
import yaml
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from segment_speed_utils.project_vars import RT_SCHED_GCS, SEGMENT_GCS
from shared_utils import portfolio_utils, rt_dates, rt_utils

In [2]:
# Times
import datetime
from loguru import logger

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
# analysis_date = rt_dates.DATES["dec2023"]

### Load in `rt_v_scheduled_trip` functions

In [5]:
dec_df = pd.read_parquet("./ah_testing_2023-12-01.parquet")

In [6]:
len(dec_df)

86128

In [7]:
dec_df.sample()

,schedule_gtfs_dataset_key,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,total_pings_for_trip,total_min_w_gtfs,total_vp,vp_in_shape
69874,7cc0cb1871dfd558f11a2885c145d144,a551d35861e1db5e3929a08593984827,71.78,71,213,72,213.00,213.00


In [8]:
nov_df = pd.read_parquet("./ah_testing_2023-11-15.parquet")

In [9]:
nov_df.sample()

,schedule_gtfs_dataset_key,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,total_pings_for_trip,total_min_w_gtfs,total_vp,vp_in_shape
57714,fb467982dcc77a7f9199bebe709bb700,5582393dc9a747106480c1c272347718,52.32,52,155,52,155.00,118.00


In [10]:
len(nov_df)

86832

### Add back routes-schedule-trip instance
* This will go into rt_v_scheduled.trip
#### Fix time_of_day buckets
* https://github.com/cal-itp/data-analyses/blob/route_agg/rt_segment_speeds/segment_speed_utils/gtfs_schedule_wrangling.py


In [11]:
def temp_function(df, analysis_date: str):
    routes_df = helpers.import_scheduled_trips(
        analysis_date,
        columns=[
            "gtfs_dataset_key",
            "route_id",
            "direction_id",
            "trip_instance_key",
        ],
        get_pandas=True,
    )

    df2 = pd.merge(
        df,
        routes_df,
        on=["schedule_gtfs_dataset_key", "trip_instance_key"],
        how="left",
        indicator="sched_rt_category",
    )

    df2 = df2.assign(
        route_id=df2.route_id.fillna("Unknown"),
        direction_id=df2.direction_id.astype("Int64"),
        total_vp = df2.total_vp.fillna(0).astype("Int64"),
        vp_in_shape = df2.vp_in_shape.fillna(0).astype("Int64"),
        sched_rt_category=df2.apply(
            lambda x: "vp_only" if x.sched_rt_category == "left_only" else "vp_sched",
            axis=1,
        ),
    )

    sched_time_of_day = gtfs_schedule_wrangling.get_trip_time_buckets(analysis_date)[
        ["trip_instance_key", "time_of_day", "service_minutes"]
    ].pipe(gtfs_schedule_wrangling.add_peak_offpeak_column)[
        ["trip_instance_key", "service_minutes", "peak_offpeak"]
    ]

    df3 = pd.merge(df2, sched_time_of_day, on="trip_instance_key", how="left")

    rt_time_of_day = gtfs_schedule_wrangling.get_vp_trip_time_buckets(analysis_date)

    df4 = pd.merge(
        df3,
        rt_time_of_day,
        on=["schedule_gtfs_dataset_key", "trip_instance_key"],
        how="inner",
    )
    df4 = df4.assign(
        peak_offpeak=df4.peak_offpeak_x.fillna(df4.peak_offpeak_y)
    )
    
    df4 = df4.drop(
            columns=["peak_offpeak_x", "peak_offpeak_y"]
        )

    return df4

In [12]:
start = datetime.datetime.now()
print(start)
nov_df2 = temp_function(nov_df, rt_dates.DATES["nov2023"])
end = datetime.datetime.now()
print(end)

2024-02-13 15:48:49.733495
2024-02-13 15:50:17.763787


In [13]:
start = datetime.datetime.now()
print(start)
dec_df2 = temp_function(dec_df, rt_dates.DATES["nov2023"])
end = datetime.datetime.now()
print(end)

2024-02-13 15:50:17.773207
2024-02-13 15:51:37.401496


### Trips: add back metrics

In [14]:
def add_metrics(df: pd.DataFrame) -> pd.DataFrame:

    df["pings_per_min"] = df.total_pings_for_trip / df.rt_service_min
    df["spatial_accuracy_pct"] = df.vp_in_shape / df.total_vp
    df["rt_w_gtfs_pct"] = df.total_min_w_gtfs / df.rt_service_min
    df["rt_v_scheduled_time_pct"] = df.rt_service_min / df.service_minutes - 1

    # Mask rt_triptime_w_gtfs_pct for any values above 100%
    df.rt_w_gtfs_pct = df.rt_w_gtfs_pct.mask(df.rt_w_gtfs_pct > 1)

    drop_cols = ["total_pings_for_trip", "vp_in_shape", "total_vp", "total_min_w_gtfs"]
    df = df.drop(columns=drop_cols)
    return df

In [15]:
dec_trip = add_metrics(dec_df2)

In [16]:
nov_trip = add_metrics(nov_df2)

In [17]:
nov_trip.sample()

,schedule_gtfs_dataset_key,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,route_id,direction_id,sched_rt_category,service_minutes,peak_offpeak,pings_per_min,spatial_accuracy_pct,rt_w_gtfs_pct,rt_v_scheduled_time_pct
48091,1fd2f07342d966919b15d5d37fda8cc8,14bbad9c93ef684f65a5686f764f174d,30.97,31,f26ddad6-5d07-4cad-aca3-0407c17e058f,<NA>,vp_sched,16.87,peak,2.94,1.00,NaN,0.84


### Routes add multiple days

In [18]:
def concatenate_trip_segment_speeds(analysis_date_list: list) -> pd.DataFrame:
    """
    Concatenate the speed-trip parquets together,
    whether it's for single day or multi-day averages.
    Add columns for peak_offpeak, weekday_weekend based
    on day of week and time-of-day.
    """
    """
    SPEED_FILE = dict_inputs["stage4"]
  
    df = pd.concat([
        pd.read_parquet(
            f"{SEGMENT_GCS}{SPEED_FILE}_{analysis_date}.parquet").assign(
            service_date = pd.to_datetime(analysis_date)
        ) for analysis_date in analysis_date_list], 
        axis=0, ignore_index = True
    )
    """
    df = pd.concat(
        [
            pd.read_parquet(f"./concat_test_{analysis_date}.parquet").assign(
                service_date=pd.to_datetime(analysis_date)
            )
            for analysis_date in analysis_date_list
        ],
        axis=0,
        ignore_index=True,
    )
    return df

In [19]:
all_routes = concatenate_trip_segment_speeds(["2023-11-15", "2023-12-01"])

In [20]:
all_routes.sched_rt_category.value_counts()

vp_sched    156167
Name: sched_rt_category, dtype: int64

In [21]:
all_routes.head()

,schedule_gtfs_dataset_key,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,total_pings_for_trip,total_min_w_gtfs,total_vp,vp_in_shape,route_id,direction_id,sched_rt_category,service_minutes,peak_offpeak,service_date
0,63029a23cb0e73f2a5d98a345c5e2e40,56f15f118776aaafbf3a1c69c5821c14,62.38,62,185,63,185.00,144.00,3428,1,vp_sched,58.00,offpeak,2023-11-15
1,63029a23cb0e73f2a5d98a345c5e2e40,4244cbaa19bdbc3f6e4cc95cb792ccb0,67.70,67,201,68,201.00,147.00,3428,1,vp_sched,58.00,offpeak,2023-11-15
2,63029a23cb0e73f2a5d98a345c5e2e40,ce51c00d412991d09ad1de4ea2715f6e,127.38,127,377,127,377.00,207.00,3428,0,vp_sched,58.00,peak,2023-11-15
3,63029a23cb0e73f2a5d98a345c5e2e40,d01f03119c56bdda01210558a6f25ec2,152.02,151,449,151,449.00,186.00,3428,0,vp_sched,58.00,peak,2023-11-15
4,63029a23cb0e73f2a5d98a345c5e2e40,90e793547709584c8921f0786f9d310f,76.30,75,227,76,227.00,124.00,3429,1,vp_sched,55.00,offpeak,2023-11-15


In [22]:
all_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156167 entries, 0 to 156166
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   schedule_gtfs_dataset_key    156167 non-null  object        
 1   trip_instance_key            156167 non-null  object        
 2   rt_service_min               156167 non-null  float64       
 3   min_w_atleast2_trip_updates  156167 non-null  int64         
 4   total_pings_for_trip         156167 non-null  int64         
 5   total_min_w_gtfs             156167 non-null  int64         
 6   total_vp                     149500 non-null  float64       
 7   vp_in_shape                  149500 non-null  float64       
 8   route_id                     156167 non-null  object        
 9   direction_id                 153098 non-null  Int64         
 10  sched_rt_category            156167 non-null  object        
 11  service_minutes           

#### Add back metrics

In [23]:
def weighted_average_function(df: pd.DataFrame, group_cols: list):
    sum_cols = [
        "total_min_w_gtfs",
        "rt_service_min",
        "total_pings_for_trip",
        "service_minutes",
        "total_vp",
        "vp_in_shape",
    ]

    count_cols = ["trip_instance_key"]
    df2 = (
        df.groupby(group_cols + ["peak_offpeak"])
        .agg({**{e: "sum" for e in sum_cols}, **{e: "count" for e in count_cols}})
        .reset_index()
    )

    df2 = df2.rename(columns={"trip_instance_key": "n_trips"})

    df2 = add_metrics(df2)

    return df2

In [24]:
all_routes.sample()

,schedule_gtfs_dataset_key,trip_instance_key,rt_service_min,min_w_atleast2_trip_updates,total_pings_for_trip,total_min_w_gtfs,total_vp,vp_in_shape,route_id,direction_id,sched_rt_category,service_minutes,peak_offpeak,service_date
12128,9809d3f8121513057bc5cb8de7b54ce2,e8d592837b2ac0027ddfc710d35b4edf,58.55,52,112,57,112.00,104.00,041-131,1,vp_sched,45.00,offpeak,2023-11-15


In [25]:
all_routes2 = weighted_average_function(
    all_routes,
    ["schedule_gtfs_dataset_key", "route_id", "direction_id", "sched_rt_category"],
)

In [26]:
all_routes2.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'direction_id',
       'sched_rt_category', 'peak_offpeak', 'rt_service_min',
       'service_minutes', 'n_trips', 'pings_per_min', 'spatial_accuracy_pct',
       'rt_w_gtfs_pct', 'rt_v_scheduled_time_pct'],
      dtype='object')

#### rt_v_scheduled_trip_time_pct -> just delete entirely?
* This is to determine trip timeliness, how much longer (or shorter) a trip took based on RT data compared to its scheduled length.

In [27]:
all_routes2.loc[all_routes2.route_id == "17"]

,schedule_gtfs_dataset_key,route_id,direction_id,sched_rt_category,peak_offpeak,rt_service_min,service_minutes,n_trips,pings_per_min,spatial_accuracy_pct,rt_w_gtfs_pct,rt_v_scheduled_time_pct
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0,vp_sched,offpeak,1539.42,1134.00,20,2.91,0.76,0.98,0.36
1,015d67d5b75b5cf2b710bbadadfb75f5,17,0,vp_sched,peak,1723.65,1380.00,24,2.84,0.90,0.96,0.25
2,015d67d5b75b5cf2b710bbadadfb75f5,17,1,vp_sched,offpeak,1433.97,1138.00,22,2.79,0.87,0.94,0.26
3,015d67d5b75b5cf2b710bbadadfb75f5,17,1,vp_sched,peak,1309.88,1190.00,22,2.80,0.95,0.95,0.10
445,1ebafaca8716652559b2017b6eedc4ef,17,0,vp_sched,peak,212.50,224.00,6,1.97,1.00,1.00,-0.05
521,239f3baf3dd3b9e9464f66a777f9897d,17,0,vp_sched,offpeak,190.10,122.00,11,1.11,0.80,0.86,0.56
522,239f3baf3dd3b9e9464f66a777f9897d,17,0,vp_sched,peak,230.85,134.00,12,1.07,0.74,0.84,0.72
523,239f3baf3dd3b9e9464f66a777f9897d,17,1,vp_sched,offpeak,147.85,171.00,9,1.23,0.74,0.93,-0.14
524,239f3baf3dd3b9e9464f66a777f9897d,17,1,vp_sched,peak,145.65,171.00,9,1.24,0.80,0.95,-0.15
1721,43d8d305ee692724a532f30ea63a1cbe,17,1,vp_sched,offpeak,2787.37,1944.00,36,1.91,0.97,0.97,0.43
